<a href="https://colab.research.google.com/github/djwashout/djwashout/blob/main/SD_NEXT_Dev_Branch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Google Colab Pro Notebook for SD.NEXT Dev Branch
- [Vladmandic sd.next](https://github.com/vladmandic/automatic),


In [ ]:
#@markdown # Connect Google Drive
from google.colab import drive
from IPython.display import clear_output
import ipywidgets as widgets
import os

def inf(msg, style, wdth): inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth));display(inf)

print("[0;33mConnecting...")
drive.mount('/content/gdrive')
clear_output()
inf('\u2714 Mounted','success', '50px')


In [ ]:
#@markdown #Install or update sd.next
update = True #@param {type:"boolean"}
import os

#First run, download or not Vladmatic:
if os.path.isdir('/content/gdrive/MyDrive/sd.next/'):
  inf('...sd.next already installed...','primary', '300px')
  !cd '/content/gdrive/MyDrive/sd.next'
else:
  inf('...sd.next not found...INSTALLING...','primary', '300px')
  os.mkdir('/content/gdrive/MyDrive/sd.next')
  !cd '/content/gdrive/MyDrive/sd.next'
  !git clone -b dev https://github.com/vladmandic/automatic.git '/content/gdrive/MyDrive/sd.next'
  inf('\u2714 Installed','success', '50px')

#update:
if (update == True):
  %cd '/content/gdrive/MyDrive/sd.next'
  !git fetch origin dev
  !git reset --hard origin/dev
  !git pull origin dev
  inf('\u2714 Reset, Pull, Checkout Dev','success', '300px')
  inf('\u2714 Updated','success', '50px')


In [ ]:
# @markdown #Install Extensions
import subprocess
import os
from IPython.display import display
from ipywidgets import widgets

extension_folder = widgets.Text(
    value='',
    placeholder='Enter the full path to the extensions folder',
    description='Extensions Folder:',
    disabled=False,
    layout={'width': '100%'}
)

custom_extensions = widgets.Textarea(
    value='',
    placeholder='Enter extension links, separated by commas',
    description='Extensions:',
    disabled=False,
    layout={'width': '100%', 'height': '100px'}
)

install_button = widgets.Button(description="Install Extensions")
output = widgets.Output()

display(extension_folder, custom_extensions, install_button, output)

def install_extensions(b):
    with output:
        output.clear_output()
        if not custom_extensions.value.strip():
            print('No extensions specified. Please enter extension links in the text area.')
            return
        if not extension_folder.value.strip():
            print('No extension folder specified. Please enter the full path to the extensions folder.')
            return
        print("Starting extension installation...")

        # Get the list of extensions from the input
        extensions = [link.strip() for link in custom_extensions.value.split(',') if link.strip()]

        # Get the extension folder path
        folder_path = extension_folder.value.strip()

        # Check if the folder exists, create it if it doesn't
        if not os.path.exists(folder_path):
            try:
                os.makedirs(folder_path)
                print(f"Created directory: {folder_path}")
            except Exception as e:
                print(f"Error creating directory {folder_path}: {e}")
                return

        # Change directory
        try:
            os.chdir(folder_path)
            print(f"Changed directory to: {folder_path}")
        except Exception as e:
            print(f"Error changing directory to {folder_path}: {e}")
            return

        for extension in extensions:
            try:
                print(f"Installing {extension}...")
                subprocess.run(['git', 'clone', extension], check=True)
                print(f"Successfully installed {extension}")
            except subprocess.CalledProcessError as e:
                print(f"Error installing {extension}: {e}")
            except Exception as e:
                print(f"Unexpected error installing {extension}: {e}")

        print("\u2714 Installation complete")

install_button.on_click(install_extensions)

# Function to display info messages (replace with your actual implementation if needed)
def inf(message, style, width):
    print(message)

In [ ]:
#@markdown #Civitai Models & Lora Downloader
import requests
import os
from urllib.parse import urlparse, parse_qs
from IPython.display import display
from ipywidgets import widgets
import time
from tqdm.notebook import tqdm

# Create input widgets
api_key_input = widgets.Text(
    description="API Key:",
    value="9228646616856d98b2288f0375979d9a",
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)
destination_folder = widgets.Text(
    description="Save Folder:",
    value="/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion/",
    style={'description_width': 'initial'},
    layout={'width': '80%'}
)
model_links = widgets.Textarea(
    description="Model Links:",
    placeholder="Paste multiple links separated by commas",
    style={'description_width': 'initial'},
    layout={'width': '80%', 'height': '100px'}
)
download_button = widgets.Button(description="Process")
output = widgets.Output()

# Display the widgets
display(api_key_input, destination_folder, model_links, download_button, output)

def extract_model_info(url):
    parsed_url = urlparse(url)
    path_parts = parsed_url.path.split('/')
    query_params = parse_qs(parsed_url.query)

    if 'api/download/models' in parsed_url.path:
        model_version_id = path_parts[-1]
        return {'type': 'direct', 'model_version_id': model_version_id}
    elif 'models' in path_parts:
        model_id = path_parts[path_parts.index('models') + 1]
        model_version_id = query_params.get('modelVersionId', [None])[0]
        return {'type': 'webpage', 'model_id': model_id, 'model_version_id': model_version_id}
    else:
        return None

def get_download_url(model_info, api_key):
    if model_info['type'] == 'direct':
        return f"https://civitai.com/api/download/models/{model_info['model_version_id']}"
    else:
        headers = {"Authorization": f"Bearer {api_key}"}
        model_id = model_info['model_id']
        url = f"https://civitai.com/api/v1/models/{model_id}"

        response = requests.get(url, headers=headers)
        response.raise_for_status()
        model_data = response.json()

        if model_info['model_version_id']:
            for version in model_data['modelVersions']:
                if str(version['id']) == model_info['model_version_id']:
                    return version['files'][0]['downloadUrl']

        return model_data['modelVersions'][0]['files'][0]['downloadUrl']

def download_model(model_link, save_folder, api_key):
    try:
        model_info = extract_model_info(model_link.strip())
        if not model_info:
            raise ValueError(f"Invalid link format: {model_link}")

        download_url = get_download_url(model_info, api_key)

        print(f"Fetching file information...")
        file_response = requests.get(download_url, stream=True)
        file_response.raise_for_status()

        content_disposition = file_response.headers.get('Content-Disposition')
        if content_disposition:
            filename = content_disposition.split('filename=')[-1].strip('"')
        else:
            filename = f"model_{model_info.get('model_version_id', 'unknown')}.safetensor"

        print(f"Downloading {filename}...")
        file_path = os.path.join(save_folder, filename)

        total_size = int(file_response.headers.get('content-length', 0))

        with open(file_path, 'wb') as f, tqdm(
            desc=filename,
            total=total_size,
            unit='iB',
            unit_scale=True,
            unit_divisor=1024,
        ) as progress_bar:
            for data in file_response.iter_content(chunk_size=1024):
                size = f.write(data)
                progress_bar.update(size)

        print(f"Successfully downloaded {filename} to {file_path}")
        return True, None

    except Exception as e:
        error_message = f"Error downloading {model_link}: {str(e)}"
        print(error_message)
        return False, error_message

def batch_process(b):
    with output:
        output.clear_output()
        api_key = api_key_input.value.strip()
        save_folder = destination_folder.value.strip()

        if not api_key:
            print("Please enter a valid API key.")
            return

        if not save_folder:
            print("Please enter a destination folder.")
            return

        links = [link.strip() for link in model_links.value.split(',') if link.strip()]

        if not links:
            print("Please enter at least one model link.")
            return

        print(f"Found {len(links)} model links.")
        print(f"Destination folder: {save_folder}")

        try:
            os.makedirs(save_folder, exist_ok=True)
        except Exception as e:
            print(f"Error creating directory {save_folder}: {e}")
            return

        print(f"\nAttempting to download {len(links)} models to {save_folder}...")

        successful_downloads = 0
        problematic_links = []

        for i, link in enumerate(links, 1):
            print(f"\nProcessing model {i} of {len(links)}")
            success, error = download_model(link, save_folder, api_key)
            if success:
                successful_downloads += 1
            else:
                problematic_links.append(link)
            time.sleep(1)  # Add a small delay between downloads

        print(f"\nDownload complete. Successfully downloaded {successful_downloads} out of {len(links)} models.")

        if problematic_links:
            print("\nThe following links had issues:")
            for link in problematic_links:
                print(f"- {link}")

            print("\nThese problematic links have been left in the input box for you to correct and retry.")

        # Update the model_links input box
        updated_links = ', '.join(problematic_links)
        model_links.value = updated_links

        if successful_downloads > 0:
            print(f"\n{successful_downloads} successful links have been removed from the input box.")

        if problematic_links:
            print("Please review and correct the remaining links, then click 'Process' again to retry.")
        else:
            print("All links processed successfully. The input box has been cleared.")

# Attach the function to the button
download_button.on_click(batch_process)

In [ ]:
#@markdown #Run sd.next
from IPython.utils import capture
import time
import sys
import fileinput
import re
import os

model_path = "/content/gdrive/MyDrive/sd/stable-diffusion-webui/models/Stable-diffusion" #@param {type:"string"}
#@markdown Insert the full path of your trained model or to a folder containing multiple models

#Launch:
%cd '/content/gdrive/MyDrive/sd.next/'

# Prepare command-line arguments
args = "--no-hashing --share"
if model_path:
    args += f" --ckpt {model_path}"

!COMMANDLINE_ARGS="{args}" REQS_FILE="requirements.txt" python launch.py
